In [1]:
import pandas as pd

# Define the file paths to your CSV files
orders_file_path = '/kaggle/input/datasett/orders_data.csv'
inventory_file_path = '/kaggle/input/datasett/inventory_data.csv'

try:
    # Load the Orders data
    df_orders = pd.read_csv(orders_file_path)
    print("Successfully loaded orders.csv")
    print("First 5 rows of the Orders DataFrame:")
    print(df_orders.head())
    print("\nOrders DataFrame Info:")
    df_orders.info()
    print("-" * 50)

    # Load the Inventory data
    df_inventory = pd.read_csv(inventory_file_path)
    print("\nSuccessfully loaded inventory.csv")
    print("First 5 rows of the Inventory DataFrame:")
    print(df_inventory.head())
    print("\nInventory DataFrame Info:")
    df_inventory.info()
    print("-" * 50)

    # Now you can proceed with the data augmentation steps
    # as discussed previously. For example:

    # 1. Extract unique item information from orders
    if not df_orders.empty:
        item_details = df_orders[['SKU', 'Item title', 'Category', 'Brand']].drop_duplicates(subset=['SKU'])

        # 2. (Optional) Augment with pricing information
        item_pricing = df_orders[['SKU', 'OriginalUnitPrice']].drop_duplicates(subset=['SKU']) # Or use 'Final UnitPrice'

        # Merge item details with pricing
        item_info_for_inventory = pd.merge(item_details, item_pricing, on='SKU', how='left')

        # 3. Merge this extracted information into the inventory dataset
        if not df_inventory.empty:
            df_inventory_augmented = pd.merge(df_inventory, item_info_for_inventory, on='SKU', how='left')
            print("\nAugmented Inventory Data (first 5 rows):")
            print(df_inventory_augmented.head())
            # Save the augmented inventory DataFrame to a CSV file
            output_file_path = '/kaggle/working/inventory_augmented.csv'  # Adjust this path as needed
            
            try:
                df_inventory_augmented.to_csv(output_file_path, index=False)
                print(f"\nAugmented Inventory DataFrame saved to {output_file_path}")
            except Exception as e:
                print(f"An error occurred while saving the file: {e}")
        else:
            print("\nInventory DataFrame is empty. Augmentation skipped.")
            df_inventory_augmented = pd.DataFrame() # Create an empty df
    else:
        print("\nOrders DataFrame is empty. Augmentation skipped.")
        df_inventory_augmented = df_inventory.copy() # Use inventory as is or an empty df if inventory is also empty


except FileNotFoundError as e:
    print(f"Error: {e}. Please make sure the CSV files are in the correct directory.")
except Exception as e:
    print(f"An error occurred: {e}")

In [2]:
!pip install vllm

In [3]:
import vllm
import kagglehub
import logging

num_attempt = 1

import re

def filt(text, max_items=10):
    """
    Extracts up to `max_items` numbered list items from the input text, removes any parenthetical content,
    and returns them as a single string separated by ' | '.
    """
    # Find all items
    items = re.findall(r'^\s*\d+\.\s*(.+)$', text, re.MULTILINE)
    # Take only the first max_items
    items = items[:max_items]
    # Remove parenthetical content from each item
    cleaned_items = [re.sub(r'\s*\(.*?\)', '', item).strip() for item in items]
    return ' | '.join(cleaned_items)

class BundleSuggester:
    def __init__(self, tensor_parallel_size=2, gpu_memory_utilization=0.95):
        # Download and load the model once
        self.model_path = kagglehub.model_download(
            'qwen-lm/qwen2.5/Transformers/14b-instruct-awq/1'
        )
        self.llm = vllm.LLM(
            self.model_path,
            quantization="awq",
            tensor_parallel_size=tensor_parallel_size,
            gpu_memory_utilization=gpu_memory_utilization,
            trust_remote_code=True,
            dtype="half",
            enforce_eager=True,
            max_model_len=2048,
            disable_log_stats=True
        )
        self.sampling_params = vllm.SamplingParams(
            n=1,
            top_k=20,
            top_p=0.8,
            temperature=0.7,
            repetition_penalty=1.05,
            skip_special_tokens=False,
            max_tokens=128,
        )

        # Prompt template
        self.prompt_template = """
You are an e-commerce bundling expert. Generate product bundle suggestions for the given item.

Consider these bundle strategies:
- Complementary items (frequently bought together)
- Thematic collections (occasion/lifestyle based)
- Do not recommend items that are the exact same type (do not recommend another lipstick if the item is a lipstick).
- Take into account the gender that is most likely to prefer the given product during recommendations.
- Do not recommend the same category more than one time.

Requirements:
- List exactly 10 generic product categories.
- Use simple category names no brands, models, or specifications (example: T-shirt, mascara, jeans, bras, etc).
- Rank by bundling potential and revenue impact.
- Output format: numbered list only

Input: {description}

Output format:
1. [Category Name]
2. [Category Name]
3. [Category Name]
...
"""
    def format_prompt(self, description: str) -> str:
        return self.prompt_template.format(description=description)

    def suggest_for_batch(self, descriptions: list[str]) -> list[str]:
        prompts = [self.format_prompt(d) for d in descriptions]
        # Batch generate
        try:
            responses = self.llm.generate(prompts, self.sampling_params, use_tqdm=False)
            outputs = []
            for resp in responses:
                text = resp.outputs[0].text.strip()
                outputs.append(filt(text))
            return outputs
        except Exception as e:
            logging.error("Error in batch generation: %s", e)
            return ["- Error generating suggestions."] * len(descriptions)

from itertools import islice

def batch_iterator(iterable, size=32):
    it = iter(iterable)
    while True:
        chunk = list(islice(it, size))
        if not chunk:
            break
        yield chunk


#model = BundleSuggester()
#out = model.predict('Hugo Boss ανδρικό μαγιό σορτς με all-over handwritten logo "Marco" Μπλε Σκούρο L')

INFO 05-24 23:43:20 [__init__.py:239] Automatically detected platform cuda.


2025-05-24 23:43:21.283546: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748130201.314825     310 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748130201.322504     310 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# out = model.predict('Hugo Boss ανδρικό μαγιό σορτς με all-over handwritten logo "Marco" Μπλε Σκούρο L')
# print(out)
# print(filt(out))

In [5]:
# df = pd.read_csv("/kaggle/input/datasett/inventory_augmented.csv")

# total = len(df)
# print(f"Starting processing {total} rows...\n")

# for idx, title in enumerate(df["Item title"], start=1):
#     compl_items.append(filt(model.predict(title)))
    
#     # Print progress every 100 rows (or on last row)
#     if idx % 10 == 0 or idx == total:
#         print(f"Processed {idx}/{total} rows")

# # Assign the new column
# df["compl_items"] = compl_items

# # Save to CSV
# df.to_csv('final_dataset.csv', index=False)

In [6]:
import pandas as pd

# Configuration
INPUT_CSV = "/kaggle/input/datasett/inventory_augmented.csv"
OUTPUT_CSV = "final_dataset.csv"
BATCH_SIZE = 32  # adjust based on VRAM

# Load data
print("Loading data...")
df = pd.read_csv(INPUT_CSV)
items = df["Item title"].tolist()

# Initialize suggester
suggester = BundleSuggester()

# Run in batches
all_suggestions = []
print(f"Processing {len(items)} items in batches of {BATCH_SIZE}...")
for idx, batch in enumerate(batch_iterator(items, BATCH_SIZE), start=1):
    batch_suggestions = suggester.suggest_for_batch(batch)
    all_suggestions.extend(batch_suggestions)
    print(f"Batch {idx}: processed {len(batch)} items (total {len(all_suggestions)}/{len(items)})")

# Attach to dataframe and save
print("Assigning suggestions and saving...")
df["compl_items"] = all_suggestions
df.to_csv(OUTPUT_CSV, index=False)
print("Done. Output saved to", OUTPUT_CSV)

Loading data...
INFO 05-24 23:43:42 [config.py:717] This model supports multiple tasks: {'classify', 'score', 'generate', 'embed', 'reward'}. Defaulting to 'generate'.
WARNING 05-24 23:43:43 [config.py:830] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 05-24 23:43:43 [arg_utils.py:1658] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 05-24 23:43:43 [config.py:1770] Defaulting to use mp for distributed inference
WARNING 05-24 23:43:43 [cuda.py:93] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 05-24 23:43:43 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5.post1) with config: model='/kaggle/input/qwen2.5/transformers/14b-instruct-awq/1', speculative_config=None, tokenizer='/kaggle/input/qwen2.5/transformers/14b-instruct-awq/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, over

2025-05-24 23:43:49.840839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748130229.862449     353 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748130229.869120     353 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


(VllmWorkerProcess pid=353) INFO 05-24 23:43:55 [multiproc_worker_utils.py:225] Worker ready; awaiting tasks
(VllmWorkerProcess pid=353) INFO 05-24 23:43:56 [cuda.py:240] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
(VllmWorkerProcess pid=353) INFO 05-24 23:43:56 [cuda.py:289] Using XFormers backend.


[W524 23:44:07.009617326 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W524 23:44:07.393421489 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W524 23:44:17.015871593 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3


INFO 05-24 23:44:27 [utils.py:1055] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=353) INFO 05-24 23:44:27 [utils.py:1055] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=353) INFO 05-24 23:44:27 [pynccl.py:69] vLLM is using nccl==2.21.5
INFO 05-24 23:44:27 [pynccl.py:69] vLLM is using nccl==2.21.5


[W524 23:44:27.026393692 socket.cpp:204] [c10d] The hostname of the client socket cannot be retrieved. err=-3


INFO 05-24 23:44:27 [custom_all_reduce_utils.py:244] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
(VllmWorkerProcess pid=353) INFO 05-24 23:44:27 [custom_all_reduce_utils.py:244] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
(VllmWorkerProcess pid=353) WARNING 05-24 23:44:27 [custom_all_reduce.py:146] Custom allreduce is disabled because your platform lacks GPU P2P capability or P2P test failed. To silence this warning, specify disable_custom_all_reduce=True explicitly.
WARNING 05-24 23:44:27 [custom_all_reduce.py:146] Custom allreduce is disabled because your platform lacks GPU P2P capability or P2P test failed. To silence this warning, specify disable_custom_all_reduce=True explicitly.
INFO 05-24 23:44:27 [shm_broadcast.py:266] vLLM message queue communication handle: Handle(local_reader_ranks=[1], buffer_handle=(1, 4194304, 6, 'psm_bdd4c5d1'), local_subscribe_addr='ipc:///tmp/3b885bec-793e-48d9-ba94-f3d

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


(VllmWorkerProcess pid=353) INFO 05-24 23:44:39 [loader.py:458] Loading weights took 11.25 seconds
INFO 05-24 23:44:39 [loader.py:458] Loading weights took 11.29 seconds
(VllmWorkerProcess pid=353) INFO 05-24 23:44:39 [model_runner.py:1140] Model loading took 4.6720 GiB and 11.483941 seconds
INFO 05-24 23:44:39 [model_runner.py:1140] Model loading took 4.6720 GiB and 11.532345 seconds
(VllmWorkerProcess pid=353) INFO 05-24 23:44:44 [worker.py:287] Memory profiling takes 4.45 seconds
(VllmWorkerProcess pid=353) INFO 05-24 23:44:44 [worker.py:287] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.95) = 14.00GiB
(VllmWorkerProcess pid=353) INFO 05-24 23:44:44 [worker.py:287] model weights take 4.67GiB; non_torch_memory takes 0.10GiB; PyTorch activation peak memory takes 0.25GiB; the rest of the memory reserved for KV Cache is 8.98GiB.
INFO 05-24 23:44:44 [worker.py:287] Memory profiling takes 4.57 seconds
INFO 05-24 23:44:44 [worker.py:287] the curr